In [53]:
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
from langchain_groq import ChatGroq


In [54]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
import os

In [56]:
def read_pdf_files(pdf_directory):
    pdf_loader = PyPDFDirectoryLoader(pdf_directory)
    documents = pdf_loader.load()
    return documents
    

In [57]:
doc = read_pdf_files("document/")
len(doc)

340

In [58]:
def chunk_data(docs,chunk_size=800, chunk_overlap=50):
   text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
   docs = text_splitter.split_documents(docs)
   return docs

In [59]:
documents = chunk_data(docs=doc)
len(documents)

1083

In [60]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [61]:
vectors = embeddings.embed_query("how ")
len(vectors)

384

In [ ]:
from pinecone import Pinecone, ServerlessSpec

key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=key)
# Create a serverless index
index_name = "millionairefastlanechatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


In [63]:
from langchain.vectorstores import Pinecone
index = Pinecone.from_documents( documents, embeddings,index_name=index_name)

In [64]:
def retrieve_query(query,k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [65]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq


groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name="llama3-groq-70b-8192-tool-use-preview",
            temperature=0.5,
    )
chain = load_qa_chain(llm,chain_type="stuff")

In [83]:
def chatbot(query):
    matching_results = retrieve_query(query)
    response = chain.run(input_documents=matching_results,question=query)
    return response

In [84]:
query = "what is fastlane roadmap.and what is the difference between fastlane roadmap and slowlane roadmap. Give strategies for fastlane roadmap.Give any story from the book"
answer = chatbot(query)
print(answer)

The Fastlane Roadmap is an alternative financial strategy that is predicated on Controllable Unlimited Leverage. It is predisposed to wealth and capable of generating "Get Rich Quick" results, not to be confused with "Get Rich Easy." The main difference between the Fastlane Roadmap and the Slowlane Roadmap is the speed at which wealth is generated. The Fastlane Roadmap focuses on creating wealth quickly, while the Slowlane Roadmap is a more traditional approach that may take decades to achieve financial freedom.

Strategies for the Fastlane Roadmap include:

1. Building a business: Starting a business can provide unlimited leverage and potential for wealth creation.
2. Investing in real estate: Real estate investing can provide passive income and leverage through financing.
3. Investing in the stock market: Investing in stocks can provide leverage and potential for wealth creation.
4. Creating intellectual property: Creating intellectual property, such as patents or copyrights, can pro